## Introduction
This part of the code assumes that the main / body / content wrapper tag is identified elsewhere. It concerns itself only with producing a data corpus from the content section of any HTML document.

In [1]:
from bs4 import BeautifulSoup
import html2model as hm
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
main_tag = 'section'
select_type = 'id' # or cls
selector = 'section__tecspec'
html_input = 'html-test-data/zephyrus.html'
product_dictionary = hm.get_body_main(main_tag, selector, html_input)
#raw_body = ''.join(html_lines).replace('\t','')
#raw_body = raw_body.replace('specCol__', '')
#raw_body = raw_body.replace('specCol ', '')
#raw_body = raw_body.replace('specCol', '')
#raw_body = raw_body.replace('specRow', '')

In [4]:
product_dictionary.keys()

dict_keys(['main_tag', 'main_div_id', 'soup_main', 'clean_soup', 'raw_html', 'body_text', 'page_title'])

In [29]:
without_format_tags = hm.unwrap_format_tags(product_dictionary['soup_main'])
cleaner = hm.tag_lines(str(without_format_tags))

In [32]:
line_list = [line for line in cleaner.splitlines() if len(line) > 0]

In [101]:
soup_list = [BeautifulSoup(line, 'html.parser') for line in line_list if line[1] != '/' and line[1] != '!']
text_list = [soup_line.text for soup_line in soup_list]
tags_list = [[tag.name for tag in soup_line.find_all()] for soup_line in soup_list]
[[tag.attrs for tag in soup_line.find_all()] for soup_line in soup_list]
soup_list[0].find_all()[0].attrs['class']

['wd__section', 'section__tecspec', 'limit__width']

In [84]:
soup = BeautifulSoup(raw_body, 'html.parser')
good_soup = hm.unwrap_format_tags(soup)
#good_soup = hm.discard_script_and_style(good_soup)
tags2lines = hm.tag_lines(str(good_soup))
html_lines = tags2lines.splitlines()
soups = [line for line in hm.soup_lines(html_lines)]
good_soup

NameError: name 'raw_body' is not defined

In [6]:
good_soup_lines = [soup_line for soup_line in soups if not soup_line.getText() == '']
good_html_lines = [str(soup_line) for soup_line in soups if not soup_line.getText() == '']
''.join(html_lines)

'one'

In [32]:
soups[0]

<section id="special-sectionOverview"></section>

In [33]:
def appraise(line_tuples):
    nesting_record = []
    nest = 0
    for tpl in line_tuples:
        line = tpl[0]
        soup = tpl[1]
        tag = soup.find()
        if line:
            if not line[1] == '/' and not line[-2] == '/':
                nest += 1
            if line[0] == '<' and line[1] == '/':
                nest -= 1
        if not tag is None:
            nesting_record.append((tag.name, nest))
        else:
            nesting_record.append(('no-tag', nest))
    return nesting_record

# soups = [line for line in hm.soup_lines(html_lines) if str(line)]
soups = [line for line in hm.soup_lines(html_lines)]
html = [str(sl) for sl in soups]
html_lines[100:200]


['</th>',
 '<td class="info">14-inch Non-glare WQHD (2560 x 1440) IPS-level panel, 60Hz, 100% sRGB, Pantone® Validated, adaptive sync14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, 120Hz, 100% sRGB, Pantone® Validated, adaptive sync14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, 60Hz, 100% sRGB, Pantone® Validated',
 '</td>',
 '<td class="info">14-inch Non-glare WQHD (2560 x 1440) IPS-level panel, 60Hz, 100% sRGB, Pantone® Validated, adaptive sync14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, up to 120Hz, 100% sRGB, Pantone® Validated, adaptive sync14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, 60Hz, 100% sRGB, Pantone® Validated',
 '</td>',
 '<td class="info">14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, up to 120Hz, 100% sRGB, Pantone® Validated, adaptive sync14-inch Non-glare Full HD (1920 x 1080) IPS-level panel, 60Hz, 100% sRGB, Pantone® Validated',
 '</td>',
 '<td class="info">14-inch Non-glare Full HD (1920 x 1080) IPS-level panel

In [34]:
zip_lines = [lines for lines in zip(html_lines, soups)]
zip_lines

[('<section id="special-sectionOverview">',
  <section id="special-sectionOverview"></section>),
 ('<!--  special-sectionOverview  @start-->',
  <!--  special-sectionOverview  @start-->),
 ('<link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/tecspec/css/tecspec.css" rel="stylesheet" type="text/css"/>',
  <link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/tecspec/css/tecspec.css" rel="stylesheet" type="text/css"/>),
 ('<link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/global-font.css" rel="stylesheet" type="text/css"/>',
  <link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/global-font.css" rel="stylesheet" type="text/css"/>),
 ('<!--[if lt IE 9]>    ', &lt;!--[if lt IE 9]&gt;    ),
 ('<link rel="stylesheet" type="text/css" href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/tecspec/v2/tecsp

In [35]:
nesting_record = appraise(zip_lines)
nesting_record

[('section', 1),
 ('no-tag', 2),
 ('link', 2),
 ('link', 2),
 ('no-tag', 3),
 ('link', 4),
 ('no-tag', 5),
 ('figure', 6),
 ('no-tag', 5),
 ('no-tag', 6),
 ('section', 7),
 ('no-tag', 8),
 ('table', 9),
 ('tbody', 10),
 ('tr', 11),
 ('th', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('no-tag', 10),
 ('tr', 11),
 ('th', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('no-tag', 10),
 ('tr', 11),
 ('th', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('no-tag', 10),
 ('tr', 11),
 ('th', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no-tag', 11),
 ('no-tag', 10),
 ('tr', 11),
 ('th', 12),
 ('no-tag', 11),
 ('td', 12),
 ('no

In [36]:
lines_nested = [line for line in zip(nesting_record, soups)]
best_lines = [line for line in lines_nested if line[0][0] == 'no-tag' and str(line[1])]

In [37]:
best_lines = [line for line in lines_nested if not line[0][0] == 'no-tag']
len(best_lines)

104

In [38]:
[line[1].contents[0] for line in best_lines]

[<section id="special-sectionOverview"></section>,
 <link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/tecspec/css/tecspec.css" rel="stylesheet" type="text/css"/>,
 <link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/global-font.css" rel="stylesheet" type="text/css"/>,
 <link href="https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/tecspec/v2/tecspec/css/tecspec-ie.css" rel="stylesheet" type="text/css"/>,
 <figure class="img__scroll-top"></figure>,
 <section class="wd__section section__tecspec limit__width" id="section__tecspec">ROG Zephyrus G14</section>,
 <table class="specTable_table"></table>,
 <tbody></tbody>,
 <tr class=""></tr>,
 <th class="info__modelname">Model</th>,
 <td class="info info__modelname">ROG Zephyrus G14 GA401IV</td>,
 <td class="info info__modelname">ROG Zephyrus G14 GA401IU</td>,
 <td class="info info__modelname">ROG Zephyrus G14 GA401II</td>,
 <td class="inf

In [39]:
attrs = [line[1].contents[0].attrs for line in best_lines if line[1].contents[0]]
def assign_class(attr):
    if 'class' in attr:
        attr.update({'class': attr['class']})
    else:
        attr.update({'class': None})
    return attr

#attrs = [assign_class(a) for a in attrs]
#text = [line[1].contents[0].text for line in best_lines]
attrs

[{'id': 'special-sectionOverview'},
 {'href': 'https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/tecspec/css/tecspec.css',
  'rel': ['stylesheet'],
  'type': 'text/css'},
 {'href': 'https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/v3/tecspec/v2/global-font.css',
  'rel': ['stylesheet'],
  'type': 'text/css'},
 {'rel': ['stylesheet'],
  'type': 'text/css',
  'href': 'https://dlcdnimgs.asus.com/websites/global/products/xccq31kw0rrpex5l/tecspec/v2/tecspec/css/tecspec-ie.css'},
 {'class': ['img__scroll-top']},
 {'class': ['wd__section', 'section__tecspec', 'limit__width'],
  'id': 'section__tecspec'},
 {'class': ['specTable_table']},
 {},
 {'class': []},
 {'class': ['info__modelname']},
 {'class': ['info', 'info__modelname']},
 {'class': ['info', 'info__modelname']},
 {'class': ['info', 'info__modelname']},
 {'class': ['info', 'info__modelname']},
 {'class': []},
 {'class': ['head']},
 {'class': ['info']},
 {'class': ['info']},
 {'class'

In [40]:
todays_set = [element for element in zip(text, nesting_record, attrs)]
# Output format = [ str, tuple, dict ]
# Output contnt = [ txt, wrap,  html-attributes]
len(todays_set)
len(attrs)

NameError: name 'text' is not defined

In [28]:
# Today's set format = [ str, tuple, dict ]
# Today's set contains = [ txt, wrap,  html-attributes]
# For example:
# ('Category', ('tr', 19), {'class': ['specCol', 'specCol__head']})
data = {
    'TXT': text,
    'NESTING': nesting_record,
    'ALL ATTRS': attrs
}
df = pd.DataFrame(data)

NameError: name 'text' is not defined

In [ ]:
df[df['CLASS']=='specColspecCol__head']

In [ ]:
def get_name(element):
    pass

names = [element[2]['class'] == 'specTable_tab_item' for element in todays_set]
variants = []
features = []
descriptions = []
data = {
    'PRODUCT_NAME': names,
    'VARIANT': variants,
    'FEATURE': features,
    'DESCRIPTION': descriptions,
}
'''
PRODUCT_NAME        VARIANT       CATEGORY             OS                                   GRAPHICS
ROG Zephyrus G14    GA401IV       [Ultra-slim Gaming]  [Windows 10 Home, Windows 10 Pr...]  6GB GDDR6 VRAM (Boost Clock: 1285MHz, 65W)
ROG Zephyrus G14    GA401IU       [Ultra-slim Gaming]  [Windows 10 Home, Windows 10 Pr...]  6GB GDDR6 VRAM (Boost Clock: 1435MHz, 60W)
ROG Zephyrus G14    GA401II       [Ultra-wide Gaming]  [Windows 10 Home, Windows 10 Pr...]  4GB GDDR6 VRAM (Boost Clock: 1585MHz, 55W)
ROG Zephyrus G14    GA401IH       [Ultra-slim Gaming]  [Windows 10 Home, Windows 10 Pr...]  4GB GDDR6 VRAM (Boost Clock: 1615MHz, 50W)
[ ... ]'''

In [ ]:
data = hm.get_data(html_lines)
df = pd.DataFrame(data)
df.dropna(axis=0, how='all', inplace=True, thresh=2)
df = df.fillna(axis=0, value='')

In [ ]:
df['CLASS'] = df['CLASS'].apply(lambda s: hm.remove_chars(s,'[]'))
df['ID'] = df['ID'].apply(lambda t: hm.replace_chars(t,'_'))
features = df[df['CLASS']=='specColspecCol__head']
features_text = features[['TXT']]
feature_list = list(features_text.itertuples(index=True, name=None))
df

In [ ]:
values = df[df['CLASS']=='itemInfo'].drop_duplicates()
values = values[['TXT']]
value_list = list(values.itertuples(index=True, name=None))

In [ ]:
description_list = []
for i,f in enumerate(feature_list):
    descriptions = []
    if i+1 < len(feature_list):
        descriptions = [v[1] for v in value_list if f[0] <= v[0] <= feature_list[i+1][0]]
        description_list.append(descriptions)
    else:
        descriptions = [v[1] for v in value_list if feature_list[i][0] < v[0]]
        description_list.append(descriptions)

feature_set = {
    'FEATURE': [f[1] for f in feature_list],
    'DESCRIPTION': description_list
}

In [ ]:
fdf = pd.DataFrame(feature_set)
fdf